## Tensorflow SSD Testing

In [ ]:
import time
from six import BytesIO
from PIL import Image
import tensorflow as tf
import numpy as np

### Download and load model

In [ ]:
!gsutil cp -r gs://tom-seldon-examples/tfhub/ssd .

In [ ]:
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('ssd/1')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

### Download and load image

In [ ]:
!gsutil cp -r gs://tom-seldon-examples/tfhub/test-images/street-image.png .

In [ ]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data)).convert('RGB')
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
image_path = 'street-image.png'
image_np = load_image_into_numpy_array(image_path)
image_np = np.expand_dims(image_np, 0)

### Test model

In [ ]:
detections = detect_fn(image_np)
detections

### Build JSON request with Tensorflow protocol

In [ ]:
image_list = image_np.tolist()
X = {
    "instances": image_list
}
X

In [ ]:
import json

with open('test.json', 'w') as fp:
    json.dump(X, fp)